# Import Libary

In [ ]:
import numpy as np
import gdown
import os
import shutil
import zipfile
import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Download dataset

In [ ]:
file_id = "1e45f1GmiKKtHkbZPg8WkyjemBVWgjfOE"

# URL berbagi dari Google Drive
url = 'https://drive.google.com/uc?id=' + file_id

# Path untuk menyimpan file yang diunduh
output = 'dataset.zip'

# Mengunduh file
gdown.download(url, output, quiet=False)

# Mengekstrak file zip jika diperlukan
import zipfile

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('datasets/')

Downloading...
From (original): https://drive.google.com/uc?id=1e45f1GmiKKtHkbZPg8WkyjemBVWgjfOE
From (redirected): https://drive.google.com/uc?id=1e45f1GmiKKtHkbZPg8WkyjemBVWgjfOE&confirm=t&uuid=2af2dcc0-7f95-409d-a583-16a60be42f9d
To: /content/dataset.zip
100%|██████████| 653M/653M [00:08<00:00, 78.1MB/s]


# Preprocessing

## Load and Clean Dataset

In [ ]:
def check_and_remove_invalid_images(root_dir):
    invalid_files = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Attempt to open and verify the image
            except Exception as e:
                print(f"Error processing image: {file_path} - {e}")
                invalid_files.append(file_path)
    # Remove invalid files
    for invalid_file in invalid_files:
        os.remove(invalid_file)
        print(f"Invalid image removed: {invalid_file}")

In [ ]:
def get_file_paths_and_labels(root_dir):
    # Check and remove invalid images before collecting file paths and labels
    check_and_remove_invalid_images(root_dir)

    file_paths = []
    labels = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            label = os.path.basename(subdir)
            file_paths.append(file_path)
            labels.append(label)
    return file_paths, labels

In [ ]:
# Direktori asal, train, dan validation
root_dir = '/content/datasets/'
train_dir = '/content/final-datasets/train'
val_dir = '/content/final-datasets/valid'

# Membuat direktori train dan validation jika belum ada
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Mendapatkan semua jalur file dan labelnya
file_paths, labels = get_file_paths_and_labels(root_dir)

Error processing image: /content/datasets/daun kemangi/daun-kemangi_160.jpg - cannot identify image file '/content/datasets/daun kemangi/daun-kemangi_160.jpg'
Error processing image: /content/datasets/daun kemangi/daun-kemangi_154.jpg - cannot identify image file '/content/datasets/daun kemangi/daun-kemangi_154.jpg'
Error processing image: /content/datasets/daun kemangi/daun-kemangi_180.jpg - cannot identify image file '/content/datasets/daun kemangi/daun-kemangi_180.jpg'
Error processing image: /content/datasets/bawang putih/bawang-putih_152.jpg - cannot identify image file '/content/datasets/bawang putih/bawang-putih_152.jpg'
Invalid image removed: /content/datasets/daun kemangi/daun-kemangi_160.jpg
Invalid image removed: /content/datasets/daun kemangi/daun-kemangi_154.jpg
Invalid image removed: /content/datasets/daun kemangi/daun-kemangi_180.jpg
Invalid image removed: /content/datasets/bawang putih/bawang-putih_152.jpg


## Split Dataset

In [ ]:
def split_and_move_files(file_paths, labels, train_dir, val_dir, test_size=0.2):
    X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=test_size, stratify=labels, random_state=234)

    for file_path, label in zip(X_train, y_train):
        label_dir = os.path.join(train_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(file_path, label_dir)

    for file_path, label in zip(X_val, y_val):
        label_dir = os.path.join(val_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(file_path, label_dir)

In [ ]:
# Membagi dan memindahkan file ke direktori train dan validation
split_and_move_files(
    file_paths,
    labels,
    train_dir,
    val_dir,
    test_size=0.2
    )

## Data Generator and Augmentation

In [ ]:
# ImageDataGenerator untuk training dan validation
train_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

# Membuat generator untuk training dan validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 5049 images belonging to 31 classes.
Found 1263 images belonging to 31 classes.


# Modeling

In [ ]:
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(31, activation='softmax')
])

optimizer = 'rmsprop'
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    batch_size = 32,
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 681ms/step - accuracy: 0.0495 - loss: 3.6499

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


158/158 ━━━━━━━━━━━━━━━━━━━━ 132s 746ms/step - accuracy: 0.0497 - loss: 3.6479 - val_accuracy: 0.0974 - val_loss: 3.4264
Epoch 2/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 114s 694ms/step - accuracy: 0.1722 - loss: 2.8178 - val_accuracy: 0.2423 - val_loss: 2.4650
Epoch 3/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 116s 698ms/step - accuracy: 0.2603 - loss: 2.4823 - val_accuracy: 0.3508 - val_loss: 2.0517
Epoch 4/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 139s 690ms/step - accuracy: 0.3235 - loss: 2.1928 - val_accuracy: 0.4252 - val_loss: 1.7738
Epoch 5/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 114s 689ms/step - accuracy: 0.3918 - loss: 1.9498 - val_accuracy: 0.4181 - val_loss: 1.8643
Epoch 6/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 141s 682ms/step - accuracy: 0.4181 - loss: 1.8535 - val_accuracy: 0.4173 - val_loss: 1.9714
Epoch 7/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 113s 693ms/step - accuracy: 0.4498 - loss: 1.7436 - val_accuracy: 0.5234 - val_loss: 1.5494
Epoch 8/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 139s 676ms/step - accuracy: 0.4898 - loss: 1.64